In [1]:
import pandas as pd
dfs = pd.read_excel(r"D:\OneDrive - NITT\Custom_Download\final data.csv", sheet_name=None)

In [3]:
df1 = dfs['Sheet1']

In [4]:
df1 = df1.dropna()

In [5]:
df = df1.groupby('tag').agg({"pattern": lambda x:list(x),
                       "response" : lambda x: list(x)
                       })

In [6]:
df = df.reset_index(level=0)

In [7]:
corpus = df.to_dict("records")

In [8]:
data = {}
data['intents'] = corpus

In [9]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["pattern"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [11]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [13]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               69248     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 28)                1820      
                                                                 
Total params: 79,324
Trainable params: 79,324
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
22/22 [============================

22/22 [==============================] - 0s 2ms/step - loss: 0.2416 - accuracy: 0.9296
Epoch 73/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1926 - accuracy: 0.9399
Epoch 74/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1780 - accuracy: 0.9457
Epoch 75/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1834 - accuracy: 0.9428
Epoch 76/200
22/22 [==============================] - 0s 2ms/step - loss: 0.2549 - accuracy: 0.9370
Epoch 77/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1571 - accuracy: 0.9487
Epoch 78/200
22/22 [==============================] - 0s 2ms/step - loss: 0.2855 - accuracy: 0.9282
Epoch 79/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1729 - accuracy: 0.9560
Epoch 80/200
22/22 [==============================] - 0s 2ms/step - loss: 0.2429 - accuracy: 0.9443
Epoch 81/200
22/22 [==============================] - 0s 2ms/step - loss: 0.2203 - accuracy: 0.9370
Epoch 82/200


22/22 [==============================] - 0s 2ms/step - loss: 0.1896 - accuracy: 0.9501
Epoch 154/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1863 - accuracy: 0.9545
Epoch 155/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1413 - accuracy: 0.9604
Epoch 156/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1848 - accuracy: 0.9531
Epoch 157/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1298 - accuracy: 0.9604
Epoch 158/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1305 - accuracy: 0.9692
Epoch 159/200
22/22 [==============================] - 0s 2ms/step - loss: 0.2327 - accuracy: 0.9487
Epoch 160/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1454 - accuracy: 0.9589
Epoch 161/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1894 - accuracy: 0.9531
Epoch 162/200
22/22 [==============================] - 0s 2ms/step - loss: 0.1483 - accuracy: 0.9619
Epoc

In [14]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]),verbose=0)[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["response"])
      break
  return result

In [15]:
# running the chatbot
while True:
    message = input("")
    if message.lower() == 'exit':
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

hi
(540,)
How are you feeling today?
i am good
(540,)
That's good to hear
thanks
(540,)
No problem.
aahh
(540,)
Sorry I'm afraid I don't follow you.
cool
(540,)
Sorry I didn't understand you.
my name
(540,)
Please accept my deepest condolences for your loss.


KeyboardInterrupt: Interrupted by user